In [3]:
import time
import hmac
import hashlib
import requests
import numpy as np
import http
import json

In [3]:
data = open('cbapi.txt', 'r').read().splitlines()

public = data[0]
passphrase = data[1]
secret = data[2]

In [9]:
class Client:
    def __init__(self):
        data = open('cbapi.txt', 'r').read().splitlines()

        self.public = data[0]
        self.passphrase = data[1]
        self.secret = data[2]

    def sign_message(self, request) -> requests.Request:
        '''Signs the request'''

        api_key = self.public
        api_secret = self.secret

        timestamp = str(int(time.time()))
        body = (request.body or b"").decode()
        url = request.path_url.split("?")[0]
        message = f"{timestamp}{request.method}{url}{body}"
        signature = hmac.new(api_secret.encode("utf-8"), message.encode("utf-8"), digestmod=hashlib.sha256).hexdigest()

        request.headers.update(
            {
                "CB-ACCESS-SIGN": signature,
                "CB-ACCESS-TIMESTAMP": timestamp,
                "CB-ACCESS-KEY": api_key,
                "Content-Type": "application/json",
            }
        )

        return request

    def get_candles(self, product: str, start: int, end: int):
        payload = {}
        resp = requests.get(
            f"https://api.coinbase.com/api/v3/brokerage/products/{product}/candles?start={start}&end={end}&granularity=ONE_MINUTE",
            params=payload, auth=self.sign_message)
        return resp.json()

    def list_accounts(self):
        payload = {}
        resp = requests.get("https://api.coinbase.com/api/v3/brokerage/accounts", params=payload,
                            auth=self.sign_message)
        return resp.json()

    def list_products(self):
        payload = {}
        resp = requests.get("https://api.coinbase.com/api/v3/brokerage/products?product_type=SPOT", params=payload,
                            auth=self.sign_message)
        return resp.json()

    def post_sell_order(self, product: str, base_size: float):
        timestamp = str(int(time.time()))
        method = "POST"
        path = "/api/v3/brokerage/orders"
        payload = json.dumps({
            "client_order_id": str(np.random.randint(2 ** 31)),
            "product_id": product,
            "side": "SELL",
            "order_configuration": {
                "market_market_ioc": {
                    "base_size": str(base_size)
                }
            }
        })
        message = f"{timestamp}{method}{path}{payload}"
        signature = hmac.new(self.secret.encode('utf-8'), message.encode('utf-8'), digestmod=hashlib.sha256).hexdigest()

        headers = {
            'CB-ACCESS-KEY': self.public,
            'CB-ACCESS-TIMESTAMP': timestamp,
            'CB-ACCESS-SIGN': signature,
            'accept': 'application/json'
        }

        conn = http.client.HTTPSConnection("api.coinbase.com")
        conn.request(method, path, payload, headers)
        res = conn.getresponse()
        data = res.read()
        print(data.decode("utf-8"))

    def post_buy_order(self, product: str, quote_size: float):
        timestamp = str(int(time.time()))
        method = "POST"
        path = "/api/v3/brokerage/orders"
        payload = json.dumps({
            "client_order_id": str(np.random.randint(2 ** 31)),
            "product_id": product,
            "side": "BUY",
            "order_configuration": {
                "market_market_ioc": {
                    "quote_size": str(quote_size)
                }
            }
        })
        message = f"{timestamp}{method}{path}{payload}"
        signature = hmac.new(self.secret.encode('utf-8'), message.encode('utf-8'), digestmod=hashlib.sha256).hexdigest()

        headers = {
            'CB-ACCESS-KEY': self.public,
            'CB-ACCESS-TIMESTAMP': timestamp,
            'CB-ACCESS-SIGN': signature,
            'accept': 'application/json'
        }

        conn = http.client.HTTPSConnection("api.coinbase.com")
        conn.request(method, path, payload, headers)
        res = conn.getresponse()
        data = res.read()
        return json.loads(data.decode("utf-8"))

In [10]:
client = Client()

In [16]:
client.post_buy_order('BTC-USD', 1000)

{'success': False,
 'failure_reason': 'UNKNOWN_FAILURE_REASON',
 'order_id': '',
 'error_response': {'error': 'INSUFFICIENT_FUND',
  'message': 'Insufficient balance in source account',
  'error_details': '',
  'preview_failure_reason': 'PREVIEW_INSUFFICIENT_FUND'},
 'order_configuration': {'market_market_ioc': {'quote_size': '1000'}}}

In [1]:
import traceback

In [23]:
import pandas as pd

In [24]:
import csv

In [6]:
data = [[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]]
df = pd.DataFrame(data)

0